# **Daily Minimum Temperatures Dataset**

## **Tải bộ dữ liệu**

In [1]:
!gdown 1wQtP2ZJt0Jp6K8LR0MYvo3xzlY_-W6Td

Downloading...
From: https://drive.google.com/uc?id=1wQtP2ZJt0Jp6K8LR0MYvo3xzlY_-W6Td
To: /home/banhmuy/zero-to-hero/AIO2025/M03/Sales+Orders.csv
100%|████████████████████████████████████████| 133k/133k [00:00<00:00, 1.03MB/s]


In [2]:
!pip install openpyxl

In [3]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill
from openpyxl import load_workbook

In [14]:
# create new xlsx  and record data from csv file
df = pd.read_csv("Sales+Orders.csv")

#3 create new workbook
wb = Workbook()
ws = wb.active
ws.title = "Sales Orders"

#4 title
header = list(df.columns)
ws.append(header)

#5 record 10 first records
for row in df.head(10).values:
    ws.append(list(row))

#6 format
header_font = Font(bold=True, color="FFFFFF")
fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
for cell in ws[1]:
    cell.font = header_font
    cell.fill = fill

#7 save excel
wb.save("sales_orders_output.xlsx")

Process data from available Excel files

In [37]:
#1 Open original Excel file
wb = load_workbook("sales_orders_output.xlsx")
ws = wb.active

#2 get header and estimate cols index
header = [cell.value for cell in ws[1]]
idx_soamount = header.index("SOAmount")
idx_last_col = len(header)

#3 add new col "SOAmount Level"
ws.cell(row=1, column=idx_last_col + 1).value = "SOAmount Level"
ws.cell(row=1, column=idx_last_col + 1).font = Font(bold=True)

#4 Go through each line and:
#  - Delete line if SOAmount <=50
#  - Assign level
#  - Color by condition
rows_to_keep = []
for i, row in enumerate(list(ws.iter_rows(min_row=2, values_only=False)), start=2):
    soamount = row[idx_soamount].value
    if not isinstance(soamount, (int, float)) or soamount <= 50:
        continue

    # Assign new cols
    level = "High" if soamount > 100 else "Medium"
    ws.cell(row=i, column=idx_last_col + 1, value=level)

    # Color line
    fill_color = "C6EFCE" if level == "High" else "FFEB9C"
    for cell in row:
        cell.fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type="solid")
    # Add new line into keeping list
    rows_to_keep.append(i)

# save file
processed_file = "sales_orders_processed.xlsx"
wb.save(processed_file)

Charts, formulas, and advanced formatting

In [44]:
from openpyxl.chart import BarChart, Reference
from openpyxl.styles import Alignment, Border, Side, Font

#1 open original file
wb = load_workbook("sales_orders_output.xlsx")
ws = wb.active

#2 Specify the number of rows + columns "SOAmount"
max_row = ws.max_row
header = [cell.value for cell in ws[1]]
idx_soamount = header.index("SOAmount") + 1 #openyxl mark number from 1

#3 Add new sum line below SOAmount col
sum_cell = f"{chr(64+idx_soamount)}{max_row+1}"
ws[sum_cell] = f"=SUM({chr(64+idx_soamount)}2:{chr(64+idx_soamount)}{max_row})"
ws[sum_cell].font = Font(bold=True)
ws[sum_cell].alignment = Alignment(horizontal="center")

# Label "Total"
ws[f"{chr(64+idx_soamount-1)}{max_row+1}"] = "TOTAL"
ws[f"{chr(64+idx_soamount-1)}{max_row+1}"].font = Font(bold=True)

#4 Create a column chart for SOAmount
chart = BarChart()
chart.title = "SOAmount by Row"
chart.y_axis.title = "Amount"
chart.x_axis.title = "Row"

data = Reference(ws, min_col=1, min_row=2, max_row=max_row)
categories = Reference(ws, min_col=1, min_row=2, max_col=max_row) #use doc no for axis x

chart.add_data(data, titles_from_data=True)
chart.set_categories(categories)

# add new chart below table
ws.add_chart(chart, f"E{max_row+3}")

#5 center and border
thin = Side(border_style="thin", color="000000")
for row in ws.iter_rows(min_row=1, max_row=max_row+1):
    for cell in row:
        cell.alignment = Alignment(horizontal="center", vertical="center")
        cell.border = Border(left=thin, right=thin, top=thin, bottom=thin)

#6 save and allow download
final_file = "sales_orders_final.xlsx"
wb.save(final_file)
